In [1]:
import pandas as pd
import numpy as np

# Tier 1 matching - 3 datasets

In [2]:
# from dfply import *
import Levenshtein as lev
from metaphone import doublemetaphone
import geopy
from geopy.distance import great_circle

In [6]:
# ! pip install metaphone

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13918 sha256=f065422030498e15fe4060fffd691e50b70179e8168e271b96bba0fb4b36d113
  Stored in directory: c:\users\yanjiacao\appdata\local\pip\cache\wheels\b2\9e\d9\26be7687b8fe36cd6cacbec34e825a3dbcd3bae54017cfb385
Successfully built metaphone


### 1. Set up parameters

In [27]:
config_gov_esri = {
    'base_set'  : 'gov',
    'target_set': 'esri',
    'spatial_join_1000_file': "gov_esri1000m.csv",
    'gps_list'  : ['E_LAT','E_LONG','G_LAT','G_LONG'],
    'name_list' : ['E_NAME','G_NAME'],
    'base_ID'   : 'G_RECORDID',
    'target_ID' : 'E_RECORDID'
}  

config_gov_yelp = {
    'base_set'  : 'gov',
    'target_set': 'yelp',
    'spatial_join_1000_file': "gov_yelp1000m.csv",
    'gps_list'  : ['Y_LAT','Y_LONG','G_LAT','G_LONG'],
    'name_list' : ['Y_NAME','G_NAME'],
    'base_ID'   : 'Y_RECORDID',
    'target_ID' : 'G_RECORDID'
}   

config_esri_yelp = {
    'base_set'  : 'esri',
    'target_set': 'yelp',
    'spatial_join_1000_file': "esri_yelp1000m.csv",
    'gps_list'  : ['Y_LAT','Y_LONG','E_LAT','E_LONG'],
    'name_list' : ['Y_NAME','E_NAME'],
    'base_ID'   : 'E_RECORDID',
    'target_ID' : 'Y_RECORDID'
}


####-----------------------------------------
#### Set up config
# config = config_gov_yelp  ### CHANGE THIS
# config = config_gov_esri  ### CHANGE THIS
config = config_esri_yelp  ### CHANGE THIS
####-----------------------------------------


input_file = config['spatial_join_1000_file']
gps_list   = config['gps_list']
name_list  = config['name_list']
base_ID    = config['base_ID']
target_ID  = config['target_ID']

gps_threshold = 100 #meters
lev_dist_threshold = 4
dm_threshold = 2

### 2. Matching 

In [28]:
#### Read Data 
#data candidates: gov_esri1000m, gov_yelp1000m, esri_yelp1000m
join_table=pd.read_csv(input_file)

#### Assign unique ID to Base-ID and Target-ID
join_table['Base_ID_int'] = join_table.groupby([base_ID], sort=False).ngroup()
join_table['Target_ID_int'] = join_table.groupby([target_ID], sort=False).ngroup()

#### Define distance functions 
def great_circle_dist(lat_a, lng_a, lat_b, lng_b, metric='meter'):
    try:
        if metric == 'meter':
            point_a = (lat_a, lng_a)
            point_b = (lat_b, lng_b)
            return great_circle(point_a, point_b).meters
        elif metric == 'km':
            point_a = (lat_a, lng_a)
            point_b = (lat_b, lng_b)
            return great_circle(point_a, point_b).km
        elif metric == 'mile':
            point_a = (lat_a, lng_a)
            point_b = (lat_b, lng_b)
            return great_circle(point_a, point_b).miles
    except:
        return np.nan

print (join_table.shape)
print (join_table[base_ID].unique().shape)

(925329, 33)
(10790,)


In [29]:
#clean white space for leading and tail of names
join_table[name_list[0]] = join_table[name_list[0]].apply(lambda x: str(x).strip())
join_table[name_list[1]] = join_table[name_list[1]].apply(lambda x: str(x).strip())

In [30]:
#### Matching Process

# Step 1 - location match: remove food outlet from base (left side) where no target food outlet were within spatial join radius (1000m)
join_table = join_table.loc[join_table.Join_Count >0]

# Step 2 - GPS match: remove target food outlets > 'gps_threshold' (100m) away
join_table['dist_dff'] = join_table.apply(lambda x: great_circle_dist(x[gps_list[2]],x[gps_list[3]],x[gps_list[0]],x[gps_list[1]],metric='meter'), axis=1) 
join_table = join_table.loc[join_table['dist_dff'] <= gps_threshold]

# Step 3 - Store name match: remove outlet with Levenshtein > 'lev_dist_threshold'
join_table['lev_dist'] = join_table.apply(lambda x: lev.distance(x[name_list[0]],  x[name_list[1]]), axis=1)
join_table = join_table.loc[join_table['lev_dist'] <= lev_dist_threshold] 

# Step 4 - DM distance match: remove outlet with DM Dist > 'dm_threshold'
join_table['DM1'] = join_table.apply(lambda x: doublemetaphone(str(x[name_list[0]]))[0], axis=1)
join_table['DM2'] = join_table.apply(lambda x: doublemetaphone(str(x[name_list[1]]))[0], axis=1)
join_table['DM_lev'] = join_table.apply(lambda x: lev.distance(x['DM1'],  x['DM2']), axis=1)
join_table = join_table.loc[join_table['DM_lev'] <= dm_threshold] 

In [31]:
#unique number of POIs after name matching
print(f"Total target POIs after Name Match: {len(join_table[target_ID].unique())}")

# duplicated POIs in the target set (right side)
print (f"Duplicated base POIs   (left side):  {join_table.duplicated('Base_ID_int').sum()}")
print (f"Duplicated target POIs (right side): {join_table.duplicated('Target_ID_int').sum()}")

print (f"Duplicated base POIs   (left side):  {join_table.duplicated(subset=base_ID, keep='first').sum()}")
print (f"Duplicated target POIs (right side): {join_table.duplicated(subset=target_ID, keep='first').sum()}")

Total target POIs after Name Match: 4242
Duplicated base POIs   (left side):  43
Duplicated target POIs (right side): 38
Duplicated base POIs   (left side):  43
Duplicated target POIs (right side): 38


In [32]:
print (join_table.shape)
print (join_table.Base_ID_int.unique().shape)

(4280, 38)
(4237,)


### 3. Cleaning duplicates of the Target food oulet  (right side)

In [33]:
#######Extra steps for Gov & ESRI Match
#extra steps for cleaning duplicates for esri gas stations and similar names using smaller lev index
if (config['base_set'] == 'gov') and (config['target_set'] == 'esri'):
    print ("Working on: Gov & ESRI")
    
    ## Drop records where NAICS=447190  (Other Gasoline Stations)
    ## Drop records where NAICS=457110/457120  (Gasoline Stations with Convenience Stores / Other Gasoline Stations)
    join_table = join_table.loc[~(join_table['E_NAICS'].astype('str').str.startswith('44719')) & ~(join_table['E_NAICS'].astype('str').str.startswith('45711'))]
    
    #step 1 (Base ): If duplicate in RECORDID, then keep the one with lower lev score
    # keep_base_IDs = join_table.groupby(['G_RECORDID'])['lev_dist'].idxmin()
    keep_base_IDs = join_table.groupby(['Base_ID_int'])['lev_dist'].idxmin()
    #step 2 (Target): If duplicate in RECORDID, then keep the one with lower lev score
    # keep_target_IDs = join_table.loc[keep_base_IDs].groupby(['E_RECORDID'])['lev_dist'].idxmin()
    keep_target_IDs = join_table.loc[keep_base_IDs].groupby(['Target_ID_int'])['lev_dist'].idxmin()
    
    #save processed data
    gov_esri_match_final = join_table.loc[keep_target_IDs]
    gov_esri_match_final.to_csv('gov_esri_tier1_for3match_20230512.csv',index=False)
    

#######Extra steps for Gov & Yelp Match
if (config['base_set'] == 'gov') and (config['target_set'] == 'yelp'):
    print ("Working on: Gov & Yelp")

    #step 1 (Base)  : If duplicate in Base RECORDID, then keep the one with lower lev score
    # keep_base_IDs   = join_table.groupby(['G_RECORDID'])['lev_dist'].idxmin()
    keep_base_IDs   = join_table.groupby(['Base_ID_int'])['lev_dist'].idxmin()
    #step 2 (Target): If duplicate in RECORDID, then keep the one with lower lev score
    # keep_target_IDs = join_table.loc[keep_base_IDs].groupby(['Y_RECORDID'])['lev_dist'].idxmin()
    keep_target_IDs = join_table.loc[keep_base_IDs].groupby(['Target_ID_int'])['lev_dist'].idxmin()
    
    #save processed data
    gov_yelp_match_final = join_table.loc[keep_target_IDs]
    gov_yelp_match_final.to_csv('gov_yelp_tier1_for3match_20230512.csv',index=False)
    
    
#######Extra steps for Esri & Yelp Match
if (config['base_set'] == 'esri') and (config['target_set'] == 'yelp'):
    print ("Working on: Esri & Yelp")
    
    #step 1 (Base)  : If duplicate in Base RECORDID, then keep the one with lower lev score
    # keep_base_IDs   = join_table.groupby(['E_RECORDID'])['lev_dist'].idxmin()
    keep_base_IDs   = join_table.groupby(['Base_ID_int'])['lev_dist'].idxmin()
    #step 2 (Target): If duplicate in Target RECORDID, then keep the one with lower lev score
    keep_target_IDs = join_table.loc[keep_base_IDs].groupby(['Y_RECORDID'])['lev_dist'].idxmin()
    keep_target_IDs = join_table.loc[keep_base_IDs].groupby(['Target_ID_int'])['lev_dist'].idxmin()
    
    #save processed data
    esri_yelp_match_final = join_table.loc[keep_target_IDs]
    esri_yelp_match_final.to_csv('esri_yelp_tier1_for3match_20230512.csv',index=False)

Working on: Esri & Yelp


In [34]:
print (f"gov_esri_match  : {gov_esri_match_final.shape[0]}")
print (f"gov_yelp_match  : {gov_yelp_match_final.shape[0]}")
print (f"esri_yelp_match : {esri_yelp_match_final.shape[0]}")

gov_esri_match  : 2688
gov_yelp_match  : 3485
esri_yelp_match : 4208


In [36]:
len(gov_yelp_match_final['G_RECORDID'].unique())

3485

### 4. Matching 3 datasets

In [4]:
gov_esri_match_final = pd.read_csv('gov_esri_tier1_for3match_20230512.csv')
gov_yelp_match_final = pd.read_csv('gov_yelp_tier1_for3match_20230512.csv')
esri_yelp_match_final = pd.read_csv('esri_yelp_tier1_for3match_20230512.csv')

In [37]:
cols_GE = ['G_NAME','G_RECORDID','E_NAME','E_RECORDID']
cols_GY = ['G_NAME','G_RECORDID','Y_NAME','Y_RECORDID']
cols_EY = ['E_NAME','E_RECORDID','Y_NAME','Y_RECORDID']

In [38]:
### gov-esri-yelp  (gov_esri + gov_yelp)
tier1_3way_v1 = gov_esri_match_final[cols_GE].merge(gov_yelp_match_final[cols_GY],on=['G_RECORDID','G_NAME'],how='inner')
print (len(tier1_3way_v1))

### gov_esri_yelp  (gov_esri + esri_yelp)
tier1_3way_v2 = gov_esri_match_final[cols_GE].merge(esri_yelp_match_final[cols_EY],on=['E_RECORDID','E_NAME'],how='inner')
print (len(tier1_3way_v2))

### gov_esri_yelp  (gov_yelp + esri_yelp)
tier1_3way_v3 = gov_yelp_match_final[cols_GY].merge(esri_yelp_match_final[cols_EY],on=['Y_RECORDID','Y_NAME'],how='inner')
print (len(tier1_3way_v3))

2069
2072
2118


In [ ]:
#merge all three combinations
tier1_3way_all=pd.concat([tier1_3way_v1, tier1_3way_v2, tier1_3way_v3], ignore_index=True)


In [40]:
### Drop duplicates if all columns are the same
tier1_3way_final = tier1_3way_all.drop_duplicates()
print (len(tier1_3way_final))

2185


### 5. Further drop duplicates coming from different sets

In [ ]:
### Check if there are duplicates based on G_RECORDID
tier1_3way_final[tier1_3way_final.duplicated(['G_RECORDID'], keep=False)].sort_values(['G_RECORDID'])

In [ ]:
### Calculate 
tier1_3way_final['lev_dist_GE'] = tier1_3way_final.copy().apply(lambda x: lev.distance(x['G_NAME'],  str(x['E_NAME'])), axis=1)
tier1_3way_final['lev_dist_GY'] = tier1_3way_final.copy().apply(lambda x: lev.distance(x['G_NAME'],  x['Y_NAME']), axis=1)
tier1_3way_final['lev_dist_EY'] = tier1_3way_final.copy().apply(lambda x: lev.distance(str(x['E_NAME']),  x['Y_NAME']), axis=1)
tier1_3way_final['lev_dist_score'] = tier1_3way_final['lev_dist_GE'] + tier1_3way_final['lev_dist_GY'] + tier1_3way_final['lev_dist_EY']

In [43]:
### Use G-E to clean Y
Y_lev = tier1_3way_final.groupby(['G_RECORDID','E_RECORDID']).lev_dist_score.transform(min)
tier1_3way_final_clean = tier1_3way_final.loc[tier1_3way_final.lev_dist_score == Y_lev]

### Use G-Y to clean E
E_lev = tier1_3way_final_clean.groupby(['G_RECORDID','Y_RECORDID']).lev_dist_score.transform(min)
tier1_3way_final_clean = tier1_3way_final_clean.loc[tier1_3way_final_clean.lev_dist_score == E_lev]

### Use E-Y to clean G
G_lev = tier1_3way_final_clean.groupby(['E_RECORDID','Y_RECORDID']).lev_dist_score.transform(min)
tier1_3way_final_clean = tier1_3way_final_clean.loc[tier1_3way_final_clean.lev_dist_score == G_lev]

In [44]:
tier1_3way_final_clean.to_csv('tier1_3match_20230512.csv',index=False)

### 6. Prepare data for Tier 1 2-way matching

In [ ]:
#Take 3 match from each of the original data, prep for 2 match
tier1_3way_final_clean=pd.read_csv('tier1_3match_20230512.csv')

#read original data
gov_all=pd.read_csv("gov_clean_final.csv")
esri_all=pd.read_csv("esri_clean_final.csv")
yelp_all=pd.read_csv("yelp_clean_final.csv")

gov_tier1_for_2match=gov_all.loc[~gov_all['G_RECORDID'].isin(tier1_3way_final_clean['G_RECORDID'])]
esri_tier1_for_2match=esri_all.loc[~esri_all['E_RECORDID'].isin(tier1_3way_final_clean['E_RECORDID'])]
yelp_tier1_for_2match=yelp_all.loc[~yelp_all['Y_RECORDID'].isin(tier1_3way_final_clean['Y_RECORDID'])]

gov_tier1_for_2match.to_csv('gov_for_tier1_2match_0512.csv')
esri_tier1_for_2match.to_csv('esri_for_tier1_2match_0512.csv')
yelp_tier1_for_2match.to_csv('yelp_for_tier1_2match_0512.csv')